<a href="https://colab.research.google.com/github/rajeevfromkrec/deep_learning/blob/master/consumer_complaint_classification_LSTM_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import zipfile
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
#from chart_studio.plotly import iplot
from plotly.offline import iplot
from keras.utils.np_utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
!wget https://files.consumerfinance.gov/ccdb/complaints.csv.zip

--2020-06-07 10:30:47--  https://files.consumerfinance.gov/ccdb/complaints.csv.zip
Resolving files.consumerfinance.gov (files.consumerfinance.gov)... 54.192.86.15, 54.192.86.106, 54.192.86.121, ...
Connecting to files.consumerfinance.gov (files.consumerfinance.gov)|54.192.86.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 260283614 (248M) [binary/octet-stream]
Saving to: ‘complaints.csv.zip’

complaints.csv.zip  100%[===================>] 248.23M  29.5MB/s    in 8.9s    

2020-06-07 10:30:56 (27.8 MB/s) - ‘complaints.csv.zip’ saved [260283614/260283614]



In [0]:
zfile=zipfile.ZipFile('complaints.csv.zip')
zfile.extractall()

In [4]:
!ls

complaints.csv	complaints.csv.zip  sample_data


In [0]:
df=pd.read_csv('complaints.csv')

In [6]:
df.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2019-09-24,Debt collection,I do not know,Attempts to collect debt not owed,Debt is not yours,transworld systems inc. \nis trying to collect...,NaN,TRANSWORLD SYSTEMS INC,FL,335XX,NaN,Consent provided,Web,2019-09-24,Closed with explanation,Yes,NaN,3384392
1,2019-09-19,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,PA,15206,NaN,Consent not provided,Web,2019-09-20,Closed with non-monetary relief,Yes,NaN,3379500
2,2019-10-25,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,I would like to request the suppression of the...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",CA,937XX,NaN,Consent provided,Web,2019-10-25,Closed with explanation,Yes,NaN,3417821
3,2019-11-08,Debt collection,I do not know,Communication tactics,Frequent or repeated calls,"Over the past 2 weeks, I have been receiving e...",NaN,"Diversified Consultants, Inc.",NC,275XX,NaN,Consent provided,Web,2019-11-08,Closed with explanation,Yes,NaN,3433198
4,2019-02-08,Vehicle loan or lease,Lease,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,NaN,NaN,HYUNDAI CAPITAL AMERICA,FL,33161,NaN,Consent not provided,Web,2019-02-08,Closed with non-monetary relief,Yes,NaN,3146310


In [7]:
df.describe(include='all')

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
count,1626134,1626134,1390969,1626133,1055272,542085,631055,1626134,1598551,1470926,220162,985732,1626134,1626134,1626133,1626134,768477,1.626134e+06
unique,3110,18,76,166,218,506331,10,5637,63,24071,3,4,6,3059,8,2,2,NaN
top,2017-09-08,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,There are many mistakes appear in my report wi...,Company has responded to the consumer and the ...,"EQUIFAX, INC.",CA,300XX,Servicemember,Consent provided,Web,2017-09-08,Closed with explanation,Yes,No,NaN
freq,3553,414783,408159,264427,148264,1035,474737,171509,219674,10629,112866,542427,1238997,3387,1273193,1589275,620099,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.252566e+06
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.060964e+06
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+00
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.409502e+06
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.566228e+06
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.145446e+06


In [8]:
df['Product'].value_counts()

Credit reporting, credit repair services, or other personal consumer reports    414783
Mortgage                                                                        302982
Debt collection                                                                 296380
Credit reporting                                                                140432
Credit card                                                                      89190
Bank account or service                                                          86206
Credit card or prepaid card                                                      78505
Checking or savings account                                                      64676
Student loan                                                                     58189
Consumer Loan                                                                    31604
Vehicle loan or lease                                                            17529
Money transfer, virtual currency, or money 

In [9]:
df=df[pd.notnull(df['Consumer complaint narrative'])]
df['Product'].value_counts()


Credit reporting, credit repair services, or other personal consumer reports    172320
Debt collection                                                                 114665
Mortgage                                                                         65292
Credit card or prepaid card                                                      37044
Credit reporting                                                                 31588
Student loan                                                                     26181
Checking or savings account                                                      21615
Credit card                                                                      18838
Bank account or service                                                          14885
Consumer Loan                                                                     9473
Vehicle loan or lease                                                             9179
Money transfer, virtual currency, or money 

In [10]:
df['Consumer complaint narrative'][0:10]

0     transworld systems inc. \nis trying to collect...
2     I would like to request the suppression of the...
3     Over the past 2 weeks, I have been receiving e...
12    I was sold access to an event digitally, of wh...
13    While checking my credit report I noticed thre...
17    Today XX/XX/XXXX went online to dispute the in...
19    XXXX is reporting incorrectly to Equifax and X...
20    Please reverse the late payments reported on t...
21    I would like the credit bureau to correct my X...
22    MY NAME IS XXXX XXXX XXXX I live at XXXX XXXX ...
Name: Consumer complaint narrative, dtype: object

In [0]:
df.loc[df['Product']== 'Credit reporting','Product']= 'Credit reporting, credit repair services, or other personal consumer reports'
df.loc[df['Product']== 'Credit card','Product']='Credit card or prepaid card'
df.loc[df['Product']== 'Prepaid card','Product']='Credit card or prepaid card'
df.loc[df['Product']== 'Payday loan','Product']='Payday loan, title loan, or personal loan'
df.loc[df['Product']== 'Virtual currency','Product']='Money transfer, virtual currency, or money service'
df=df[df.Product !='Other financial service' ]

In [12]:
df['Product'].value_counts()

Credit reporting, credit repair services, or other personal consumer reports    203908
Debt collection                                                                 114665
Mortgage                                                                         65292
Credit card or prepaid card                                                      57332
Student loan                                                                     26181
Checking or savings account                                                      21615
Bank account or service                                                          14885
Consumer Loan                                                                     9473
Vehicle loan or lease                                                             9179
Payday loan, title loan, or personal loan                                         8963
Money transfer, virtual currency, or money service                                8803
Money transfers                            

In [0]:
df = df.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
df['Consumer complaint narrative'] = df['Consumer complaint narrative'].apply(clean_text)

In [0]:
vocab_size=50000
seq_length=100
embedded_dim=100

tokenizer= Tokenizer(num_words=vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_{|}~',lower=True)



In [0]:
tokenizer.fit_on_texts(df['Consumer complaint narrative'].values)

In [0]:
word_index=tokenizer.word_index

In [17]:
len(word_index)

191287

In [18]:
word_index

{'credit': 1,
 'account': 2,
 'report': 3,
 'information': 4,
 'would': 5,
 'payment': 6,
 'loan': 7,
 'debt': 8,
 'bank': 9,
 'told': 10,
 'received': 11,
 'company': 12,
 'card': 13,
 'time': 14,
 'called': 15,
 'reporting': 16,
 'never': 17,
 'payments': 18,
 'sent': 19,
 'letter': 20,
 'also': 21,
 'back': 22,
 'pay': 23,
 'get': 24,
 'paid': 25,
 'call': 26,
 'mortgage': 27,
 'said': 28,
 'accounts': 29,
 'made': 30,
 'amount': 31,
 'one': 32,
 'due': 33,
 'number': 34,
 'could': 35,
 'phone': 36,
 'days': 37,
 'money': 38,
 'balance': 39,
 'consumer': 40,
 'collection': 41,
 'late': 42,
 'still': 43,
 'asked': 44,
 'since': 45,
 'date': 46,
 'nt': 47,
 'even': 48,
 'please': 49,
 'dispute': 50,
 'years': 51,
 'name': 52,
 'file': 53,
 'make': 54,
 'request': 55,
 'contacted': 56,
 'home': 57,
 'us': 58,
 'check': 59,
 'interest': 60,
 'month': 61,
 'service': 62,
 'removed': 63,
 'months': 64,
 'new': 65,
 'agency': 66,
 'complaint': 67,
 'help': 68,
 'times': 69,
 'address': 70,

In [19]:
print( 'found %s unique tokens.' %len(word_index))

found 191287 unique tokens.


In [0]:
X = tokenizer.texts_to_sequences(df['Consumer complaint narrative'].values)
X = pad_sequences(X, maxlen=seq_length)

In [21]:
X.shape

(541793, 100)

In [22]:
Y = pd.get_dummies(df['Product']).values
Y.shape

(541793, 12)

In [23]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(487613, 100) (487613, 12)
(54180, 100) (54180, 12)


In [24]:
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Embedding(vocab_size,embedded_dim,input_length=X.shape[1]),
                                    tf.keras.layers.SpatialDropout1D(0.2),
                                    tf.keras.layers.LSTM(100,dropout=0.2,recurrent_dropout=0.2,activation='relu'),
                                    tf.keras.layers.Dense(12,activation='softmax')


])

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 100)          5000000   
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 100, 100)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 12)                1212      
Total params: 5,081,612
Trainable params: 5,081,612
Non-trainable params: 0
_________________________________________________________________


In [26]:
epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Epoch 1/5
6858/6858 [==============================] - 2672s 390ms/step - loss: 13.1213 - accuracy: 0.3713 - val_loss: 1.8300 - val_accuracy: 0.3756
Epoch 2/5
6858/6858 [==============================] - 2691s 392ms/step - loss: 1.9510 - accuracy: 0.3772 - val_loss: 1.7502 - val_accuracy: 0.3769
Epoch 3/5
6858/6858 [==============================] - 2693s 393ms/step - loss: 1.6076 - accuracy: 0.4282 - val_loss: 1.3981 - val_accuracy: 0.4931
Epoch 4/5
6858/6858 [==============================] - 2667s 389ms/step - loss: 1.1963 - accuracy: 0.6138 - val_loss: 0.8965 - val_accuracy: 0.7342
Epoch 5/5
6858/6858 [==============================] - 2666s 389ms/step - loss: 0.7103 - accuracy: 0.7757 - val_loss: 0.6069 - val_accuracy: 0.8026


In [0]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [0]:
plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show();

In [0]:
plt.title('Accuracy')
plt.plot(history.history['acc'], label='train')
plt.plot(history.history['val_acc'], label='test')
plt.legend()
plt.show();

In [0]:
new_complaint = ['I am a victim of identity theft and someone stole my identity and personal information to open up a Visa credit card account with Bank of America. The following Bank of America Visa credit card account do not belong to me : XXXX.']
seq = tokenizer.texts_to_sequences(new_complaint)
padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
pred = model.predict(padded)
labels = ['Credit reporting, credit repair services, or other personal consumer reports', 'Debt collection', 'Mortgage', 'Credit card or prepaid card', 'Student loan', 'Bank account or service', 'Checking or savings account', 'Consumer Loan', 'Payday loan, title loan, or personal loan', 'Vehicle loan or lease', 'Money transfer, virtual currency, or money service', 'Money transfers', 'Prepaid card']
print(pred, labels[np.argmax(pred)])